In [ ]:
#import libraries
import time
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

Scrape 2 pages of the reviews for a single product listing using product URL.<br /> Note: Product must have more than 1 review page. The code must be generic and should work for every product listing URL, with proper checks in place.

In [ ]:
url = 'https://www.daraz.pk/products/100-waterproof-washing-machine-cover-top-loader-automatic-semi-auto-single-bowl-all-brands-supported-durable-oxford-polyester-washer-and-dryer-covers-for-sheds-garages-balcony-blue-7kg8kg9kg10kg11kg12kg13kg15kg-i250273365-s1468026978.html?'
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(url)
driver.maximize_window()

reviewlist = []
names = []

for x in range(2):

    review = driver.find_elements(By.CLASS_NAME,'item')
    name = driver.find_elements(By.CLASS_NAME,'middle')

    for i in name:
        names.append(i.text[3:-17])

    for i in review:
        reviewlist.append(i.driver.find_elements(By.CLASS_NAME,'item-content').driver.find_elements(By.CLASS_NAME,'content').text.replace('\n',''))    

    driver.find_element(By.XPATH,'//*[@id="module_product_review"]/div/div[3]/div[2]/div/button[2]').click()
    time.sleep(5)

df = pd.DataFrame({'Name':names,'Review':reviewlist})
df

# Question 2

Scrape 80 items for a keyword search e.g., 'mobile covers'<br/>Note: Must use "click()" function of webElement, URL changing for pages is not allowed. The code must be generic and should work for every keyword, with proper checks in place. Bonus for also scraping country and Daraz Mall availability.

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
query = 'mobile cover'
driver.get(f'https://www.daraz.pk/catalog/?from=input&q={query}')
driver.maximize_window()

In [ ]:
pricelist = []
namelist = []
norevlist = []
starslist = []

countrylist = []                #Bonus


def extract_data():
    productdetails = driver.find_elements_by_class_name('c16H9d')
    product = driver.find_elements_by_class_name('c2prKC')

    count = 0
    
    driver.execute_script("window.scrollTo(0, window.scrollY + 100)")
    
    for i in range(len(product)):

        productdetails = driver.find_elements_by_class_name('c16H9d')

        productdetails[i].find_element_by_css_selector('a').click()
        time.sleep(1)
        
        starslist.append(driver.find_element_by_class_name('score-average').text)
        
        driver.execute_script("window.history.go(-1)")
        time.sleep(1)
        
        product = driver.find_elements_by_class_name('c2prKC')

        pricelist.append(product[i].find_element_by_class_name('c13VH6').text.replace("Rs. ",""))
        namelist.append(product[i].find_element_by_class_name('c16H9d').text)
        norevlist.append(product[i].find_element_by_class_name('c3XbGJ').text.replace("(","").replace(")",""))
        countrylist.append(product[i].find_element_by_class_name('c2i43- ').text)

        count+=1
        if count == 4:
            driver.execute_script("window.scrollTo(0, window.scrollY + 450)")
            count = 0

extract_data()
driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[1]/div/div[1]/div[3]/div/ul/li[9]/a').click()
time.sleep(5)
extract_data()




In [ ]:
df = pd.DataFrame({'Name':namelist,'Price':pricelist,'Stars':starslist,'Review':norevlist, 'Country':countrylist})
df

# Question 3

Which Products are on Flash sale? Display Product name, Actual Price, Discounted Price, Discount Percentage, Rating, No. of Reviews, Top 3 reviews displayed.

Display output must be like this. Products may vary.

In [ ]:
url = 'https://www.daraz.pk/'
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(url)
driver.maximize_window()


In [ ]:
productname = []
actualprice = []
dispric = []
disprc = []

driver.execute_script("window.scrollTo(0, window.scrollY + 650)")
driver.refresh()
driver.find_element_by_xpath('//*[@id="hp-flash-sale"]/div[2]/div[1]/a').click()
time.sleep(2)

flashsaleframe = driver.find_element_by_class_name('item-list-content')
flashsaleproducts = WebDriverWait(flashsaleframe, 5).until( EC.visibility_of_all_elements_located((By.CLASS_NAME,"unit-content")) )

for i in flashsaleproducts:
    productname.append(i.find_element_by_class_name('sale-title').text)
    temp = i.find_element_by_class_name('origin-price').text
    disprc.append(temp[-4:-1].replace("-",""))
    actualprice.append(i.find_element_by_class_name('origin-price-value').text.replace("Rs.",""))

    dispric.append(i.find_element_by_class_name('sale-price').text.replace("Rs.",""))

In [ ]:
df = pd.DataFrame({'Name':productname,'Actual Price':actualprice,'Discounted Price':dispric,'Discounted Percentage':disprc})


In [ ]:
df.to_csv("Q3.csv",index=False)
pd.read_csv("Q3.csv")